In [1]:
%pylab inline
import sys
sys.path.insert(0, '..')

Populating the interactive namespace from numpy and matplotlib


## Loading data

In [ ]:
# We need some data

## Network definition

In [2]:
import  tensorflow as tf
from fluidflow.nets import  dense_encoder, dense_decoder
from fluidflow.bijections import Orthogonal
from fluidflow.optimizers import RMSpropNatGrad


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [10]:
hidden_size = 10
tf.reset_default_graph()

x = tf.placeholder(shape=(None, 10), dtype=tf.float32)
x2 = tf.placeholder(shape=(None, 10), dtype=tf.float32)

bijection = Orthogonal(hidden_size, hidden_size)
eigv = tf.get_variable(name='eigen_values',
                       initializer=tf.ones(hidden_size, dtype=tf.float32))

# Encode the data to  lower dimensionality space
z1 = dense_encoder(x, latent_size=hidden_size)

# Apply the orthogonal transform
h = bijection.forward(z1)
# Apply a vector of eigen values
h= h * eigv
# Apply the transpose of the orhogonal transform
z2 = bijection.inverse(h)

xrec = dense_decoder(z1)

# Predicting next time step from residual unit
xpred = dense_decoder(z1 + z2)

In [11]:
loss = tf.reduce_mean( tf.nn.l2_loss(xrec - x) + tf.nn.l2_loss(xpred - x2))

In [13]:
global_step = tf.train.get_or_create_global_step()

optimizer = RMSpropNatGrad(learning_rate=0.001,
                           global_step=global_step)

opt_op = optimizer.minimize(loss)

Appling an orthogonality preserving step to Orthogonal//orthogonal_stiefel/A:0
Appling standard rmsprop to eigen_values:0
Appling standard rmsprop to encoder/dense/kernel:0
Appling standard rmsprop to encoder/dense/bias:0
Appling standard rmsprop to encoder/dense_1/kernel:0
Appling standard rmsprop to encoder/dense_1/bias:0
Appling standard rmsprop to encoder/dense_2/kernel:0
Appling standard rmsprop to encoder/dense_2/bias:0
Appling standard rmsprop to decoder/dense/kernel:0
Appling standard rmsprop to decoder/dense/bias:0
Appling standard rmsprop to decoder/dense_1/kernel:0
Appling standard rmsprop to decoder/dense_1/bias:0
Appling standard rmsprop to decoder/dense_2/kernel:0
Appling standard rmsprop to decoder/dense_2/bias:0


## Training

In [ ]:
# just got to do some training now